In [1]:
## IMPORT MODULES
import pandas as pd
import numpy as np
import os,time,fnmatch

In [2]:
## IMPORT PLOTTING MODULES
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import cm
%matplotlib widget


In [3]:
## CREATE LINK TO DATAPATH MODULE
import sys
sys.path.insert(0,'/home/bhabas/catkin_ws/src/crazyflie_simulation/src/crazyflie_gazebo_sim/src/utility')
from data_analysis import DataFile
dataPath = '/home/bhabas/catkin_ws/src/crazyflie_simulation/local_files/data/'

In [4]:
## CREATE AN ARRAY FOR ALL TEST COMBINATIONS
temp_list = []
for vz_d in np.arange(1.5,4.0,0.5): # [1.5,3.5]
    for vx_d in np.arange(0,3.5,0.5):   # [0,3.0]
        # print(f"{vz_d}|{vx_d}")
        temp_list.append([vz_d,vx_d])
test_arr = np.asarray(temp_list)
test_arr

array([[1.5, 0. ],
       [1.5, 0.5],
       [1.5, 1. ],
       [1.5, 1.5],
       [1.5, 2. ],
       [1.5, 2.5],
       [1.5, 3. ],
       [2. , 0. ],
       [2. , 0.5],
       [2. , 1. ],
       [2. , 1.5],
       [2. , 2. ],
       [2. , 2.5],
       [2. , 3. ],
       [2.5, 0. ],
       [2.5, 0.5],
       [2.5, 1. ],
       [2.5, 1.5],
       [2.5, 2. ],
       [2.5, 2.5],
       [2.5, 3. ],
       [3. , 0. ],
       [3. , 0.5],
       [3. , 1. ],
       [3. , 1.5],
       [3. , 2. ],
       [3. , 2.5],
       [3. , 3. ],
       [3.5, 0. ],
       [3.5, 0.5],
       [3.5, 1. ],
       [3.5, 1.5],
       [3.5, 2. ],
       [3.5, 2.5],
       [3.5, 3. ]])

In [5]:
## CREATE DATAFRAME FROM ALL TESTS

d = []                                                  # Create empty list
for vz_d,vx_d in test_arr:                              # Iter over all combinations in test_arr
    for file in os.listdir(dataPath):                   # Iter over all files in dir 
        csv = os.path.join(dataPath,file)               #   and join name w/ base path

        if fnmatch.fnmatch(file,f"Vz_{vz_d}--Vx_{vx_d}--trial_*.csv"): # If filename matched selected IC 

            trial = DataFile(csv)                       # Create object called trial

            trial_num = file[-5]                        # Retrieve trial num
            landing_rate = trial.landing_rate()         # Retrieve landing success rate
            policy,sigma = trial.grab_finalPolicy()     # Retrieve final policy and sigma
            omega_d = trial.grab_omega_d_trial()
            omega_flip = trial.grab_omega_flip_trial()
            vel_flip = trial.grab_vel_flip_trial()

            RREV_trigger,G1,G2 = policy                 # Break up policy
            RREV_sig,G1_sig,G2_sig = sigma              # Break up sigma
            wx_d,wy_d,wz_d = omega_d
            wx_f,wy_f,wz_f = omega_flip
            vx_f,vy_f,vz_f = vel_flip


            d.append((vz_d,vx_d,trial_num,landing_rate, # Append values to list
                RREV_trigger,G1,G2,
                RREV_sig,G1_sig,G2_sig,
                wx_d,wy_d,wz_d,
                wx_f,wy_f,wz_f,
                vx_f,vy_f,vz_f))

policy_df = pd.DataFrame(d, columns=( # Convert list to pd.dataframe
    'vz_d','vx_d','trial_num','landing_rate',
    'RREV_trigger','G1','G2',
    'RREV_sig','G1_sig','G2_sig',
    'wx_d','wy_d','wz_d',
    'wx_f','wy_f','wz_f',
    'vx_f','vy_f','vz_f'))
policy_df

,vz_d,vx_d,trial_num,landing_rate,RREV_trigger,G1,G2,RREV_sig,G1_sig,G2_sig,wx_d,wy_d,wz_d,wx_f,wy_f,wz_f,vx_f,vy_f,vz_f
0,1.5,0.0,1,0.00,4.11,6.18,3.78,0.07,0.38,0.28,0.0,1.594084,0.0,-0.051048,1.261809,-2.016673,0.023086,-0.008365,1.083670
1,1.5,0.0,0,0.00,5.00,7.59,3.88,0.83,0.27,0.11,0.0,5.917062,0.0,-0.567061,-2.006484,-1.838774,-0.072994,0.098063,0.824812
2,1.5,0.0,2,0.00,4.82,5.82,3.44,0.08,0.15,0.28,0.0,6.118875,0.0,-0.573339,2.427546,-2.339939,0.045777,0.013838,0.857656
3,1.5,0.5,2,0.00,4.33,7.39,3.25,0.10,0.19,0.07,0.0,-26.630638,0.0,0.108152,-16.009908,-3.229212,0.506836,-0.005285,1.166715
4,1.5,0.5,1,0.00,4.55,6.98,3.86,0.13,0.10,0.14,0.0,-24.938559,0.0,-0.118284,-15.512765,-2.237243,0.480374,0.005073,1.101052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,3.5,2.5,1,0.75,5.16,7.81,1.81,0.16,0.05,0.52,0.0,-33.333030,0.0,-0.528365,-23.278037,-2.163531,2.266449,-0.016500,2.702471
101,3.5,2.5,2,0.35,4.82,8.09,1.71,0.08,0.23,0.17,0.0,-33.203274,0.0,-0.072532,-20.304746,-1.529446,2.178745,-0.015988,2.567516
102,3.5,3.0,2,0.60,5.16,8.27,4.02,0.13,0.07,0.01,0.0,-22.577884,0.0,0.271152,-24.562926,-1.931681,2.575116,-0.059825,2.453206
103,3.5,3.0,1,0.90,5.12,6.52,0.61,0.03,0.20,0.29,0.0,-31.344009,0.0,-2.375304,-23.671941,-0.412886,2.531474,-0.078620,2.425725


In [6]:
policy_df

,vz_d,vx_d,trial_num,landing_rate,RREV_trigger,G1,G2,RREV_sig,G1_sig,G2_sig,wx_d,wy_d,wz_d,wx_f,wy_f,wz_f,vx_f,vy_f,vz_f
0,1.5,0.0,1,0.00,4.11,6.18,3.78,0.07,0.38,0.28,0.0,1.594084,0.0,-0.051048,1.261809,-2.016673,0.023086,-0.008365,1.083670
1,1.5,0.0,0,0.00,5.00,7.59,3.88,0.83,0.27,0.11,0.0,5.917062,0.0,-0.567061,-2.006484,-1.838774,-0.072994,0.098063,0.824812
2,1.5,0.0,2,0.00,4.82,5.82,3.44,0.08,0.15,0.28,0.0,6.118875,0.0,-0.573339,2.427546,-2.339939,0.045777,0.013838,0.857656
3,1.5,0.5,2,0.00,4.33,7.39,3.25,0.10,0.19,0.07,0.0,-26.630638,0.0,0.108152,-16.009908,-3.229212,0.506836,-0.005285,1.166715
4,1.5,0.5,1,0.00,4.55,6.98,3.86,0.13,0.10,0.14,0.0,-24.938559,0.0,-0.118284,-15.512765,-2.237243,0.480374,0.005073,1.101052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,3.5,2.5,1,0.75,5.16,7.81,1.81,0.16,0.05,0.52,0.0,-33.333030,0.0,-0.528365,-23.278037,-2.163531,2.266449,-0.016500,2.702471
101,3.5,2.5,2,0.35,4.82,8.09,1.71,0.08,0.23,0.17,0.0,-33.203274,0.0,-0.072532,-20.304746,-1.529446,2.178745,-0.015988,2.567516
102,3.5,3.0,2,0.60,5.16,8.27,4.02,0.13,0.07,0.01,0.0,-22.577884,0.0,0.271152,-24.562926,-1.931681,2.575116,-0.059825,2.453206
103,3.5,3.0,1,0.90,5.12,6.52,0.61,0.03,0.20,0.29,0.0,-31.344009,0.0,-2.375304,-23.671941,-0.412886,2.531474,-0.078620,2.425725


In [8]:
## POLICY AVERAGED DATAFRAME

# Group dataframe values by IC and find avg/std of their values 
policy_avg_df = policy_df.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()
policy_avg_df


vz_d vx_d landing_rate           RREV_trigger                   G1  \
                     mean       std         mean       std       mean   
0   1.5  0.0     0.000000  0.000000     4.643333  0.470567   6.530000   
1   1.5  0.5     0.000000  0.000000     4.376667  0.155349   7.506667   
2   1.5  1.0     0.016667  0.028868     4.726667  0.513939   6.460000   
3   1.5  1.5     0.000000  0.000000     4.460000  0.200000  10.226667   
4   1.5  2.0     0.000000  0.000000     3.220000  0.644903   3.470000   
5   1.5  2.5     0.000000  0.000000     2.333333  0.265016   5.586667   
6   1.5  3.0     0.000000  0.000000     2.846667  1.591048   5.870000   
7   2.0  0.0     0.016667  0.028868     5.120000  0.756637   6.483333   
8   2.0  0.5     0.000000  0.000000     4.436667  0.536315   6.936667   
9   2.0  1.0     0.133333  0.076376     4.966667  0.209841   8.013333   
10  2.0  1.5     0.233333  0.076376     4.850000  0.230000   7.793333   
11  2.0  2.0     0.316667  0.028868     5.036667  0.161658  10.813333   
12  2.0  2.5     0.233333  0.152753     4.843333  0.232451   5.470000   
13  2.0  3.0     0.116667  0.115470     5.023333  0.335012   3.800000   
14  2.5  0.0     0.183333  0.175594     4.990000  0.088882   8.496667   
15  2.5  0.5     0.200000  0.000000     5.100000  0.176918   6.890000   
16  2.5  1.0     0.716667  0.125831     5.316667  0.095044   7.316667   
17  2.5  1.5     0.833333  0.104083     5.400000  0.017321   7.690000   
18  2.5  2.0     0.283333  0.448144     4.896667  0.627402   6.780000   
19  2.5  2.5     0.650000  0.278388     5.043333  0.372335   8.616667   
20  2.5  3.0     0.716667  0.152753     5.193333  0.030551   6.403333   
21  3.0  0.0     0.333333  0.028868     4.896667  0.127410   7.770000   
22  3.0  0.5     0.500000  0.100000     4.893333  0.176163   7.400000   
23  3.0  1.0     0.816667  0.144338     4.903333  0.085049   7.743333   
24  3.0  1.5     0.866667  0.076376     5.113333  0.200333   6.626667   
25  3.0  2.0     0.783333  0.104083     5.230000  0.174356   8.813333   
26  3.0  2.5     0.716667  0.230940     5.266667  0.200083   8.463333   
27  3.0  3.0     0.616667  0.076376     5.370000  0.180278   7.316667   
28  3.5  0.0     0.533333  0.076376     4.666667  0.111505   8.573333   
29  3.5  0.5     0.600000  0.086603     4.806667  0.081445   8.270000   
30  3.5  1.0     0.700000  0.050000     4.923333  0.151767   6.903333   
31  3.5  1.5     0.900000  0.050000     5.230000  0.121655  11.450000   
32  3.5  2.0     0.883333  0.160728     5.133333  0.277549   9.190000   
33  3.5  2.5     0.650000  0.264575     4.923333  0.205508   7.490000   
34  3.5  3.0     0.816667  0.189297     5.093333  0.083267   8.306667   

                    G2            ...       wy_f                 wz_f  \
         std      mean       std  ...       mean        std      mean   
0   0.935468  3.700000  0.230651  ...   0.560957   2.298598 -2.065129   
1   0.593661  2.800000  1.342796  ... -16.084634   0.612660 -2.799901   
2   5.671517  3.246667  0.875233  ...  -5.491657  16.028165 -1.894050   
3   8.137840  1.370000  1.988492  ... -15.575749   0.994363 -2.003301   
4   2.911563  7.646667  3.037636  ...  15.785670   3.233447 -1.734951   
5   2.476900  6.976667  3.007396  ...  13.027271   3.800369 -1.140795   
6   0.153948  3.520000  0.854927  ...   5.600963   0.944873  0.094794   
7   1.401761  6.633333  4.784740  ...   1.773749   2.951251 -1.626546   
8   1.269501  1.940000  2.014274  ... -13.292631   1.005389 -2.488981   
9   1.054625  2.950000  0.797308  ... -11.292908   1.619217 -0.993884   
10  1.029093  0.826667  0.265769  ... -13.264592   2.349401 -1.175714   
11  4.143445  6.813333  5.565270  ...  -4.790506  15.978690 -0.883217   
12  4.060702  3.916667  4.900228  ...  -5.608518  14.862671 -2.106162   
13  2.131267  5.763333  0.884609  ...   8.228886   5.297130 -3.690364   
14  1.696654  3.483333  1.068847  ...   0.727938   3.419834 -3.548738   
15  1.049714  2.050000  1.299654  ... -17.646727   0.578895 

# Landing Rate Data

In [9]:
## LANDING RATE DATA
LR_df = policy_avg_df[['vz_d','vx_d','landing_rate']]
# LR_df

In [11]:
# LR_df['landing_rate']
LR_df['landing_rate','mean']


## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.plot_trisurf(LR_df['vx_d'],LR_df['vz_d'],LR_df['landing_rate','mean'],cmap=cm.jet)

ax.set_xlabel('vx_d [m/s]')
ax.set_ylabel('vz_d [m/s]')
ax.set_zlabel('Avg LR [%]')
ax.set_title('Average Landing Rate for Final Two Episodes')



ax.yaxis.set_ticks(np.arange(1.5,4.0,0.5))
fig.tight_layout()
plt.show()

## Disregard any data Vz_d < 2.5

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
## LANDING RATE PER TRIAL

fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(policy_df['vx_d'],policy_df['vz_d'],policy_df['landing_rate'])

ax.set_title('Landing Rate per Trial (Raw)')
ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('LR %')

ax.yaxis.set_ticks(np.arange(1.5,4.0,0.5))
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
## DROP VZ_D < 2.5 M/S
policy_df = policy_df[ policy_df['vz_d'] >= 2.5]                # Only keep rows w/ vz_d >= 2.5 m/s
policy_avg_df = policy_avg_df[ policy_avg_df['vz_d'] >= 2.5]    # Only keep rows w/ vz_d >= 2.5 m/s

# policy_avg_df

## RREV_trigger Avg(3D)

# RREV Data

In [47]:
## RREV_trigger Average (3D)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## RREV_TRIGGER DATA
RREV_df = policy_avg_df[['vz_d','vx_d','RREV_trigger']]
 RREV_TRIGGER PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(RREV_df['vx_d'],RREV_df['vz_d'],RREV_df['RREV_trigger','mean'],color='red')


ax.set_title('Avg RREV_trigger per Trial')
ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('RREV')

ax.yaxis.set_ticks(np.arange(2.5,4.0,0.5))
ax.set_zlim(0)
plt.show()



## RREV_trigger Raw (3D)

In [48]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(policy_df['vx_d'],policy_df['vz_d'],policy_df['RREV_trigger'])

ax.set_title('RREV_trigger per Trial (Raw)')
ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('RREV')
ax.set_zlim(0)
ax.yaxis.set_ticks(np.arange(2.5,4.0,0.5))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## RREV_trigger Std (2D)

In [109]:
for vz in [2.5,3.0,3.5]:
    df = policy_avg_df[policy_avg_df.vz_d==vz]

    fig = plt.figure()
    ax = fig.add_subplot(111)
    std = df['RREV_trigger','std']
    ax.errorbar(df['vx_d'],df['RREV_trigger','mean'],yerr=std,linestyle='None',marker='o', capsize=5, capthick=1, ecolor='black')

    ax.set_ylim(0,10)
    ax.set_xlabel('Vx_d [m/s]')
    ax.set_ylabel('RREV [rad/s]')
    ax.set_title(f'RREV vs Vx_d | Vz_d = {vz} [m/s]')
    ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# G1 Data

In [49]:
## G1 AVG DATA
G1_df = policy_avg_df[['vz_d','vx_d','G1']]
G1_df


fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(G1_df['vx_d'],G1_df['vz_d'],G1_df['G1','mean'],color='red')
ax.set_title('G1 per Trial (Avg)')
ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('G1')
ax.set_zlim(0)

ax.yaxis.set_ticks(np.arange(2.5,4.0,0.5))
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(policy_df['vx_d'],policy_df['vz_d'],policy_df['G1'])

ax.set_title('G1 per Trial (Raw)')
ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('G1')
ax.set_zlim(0)

ax.yaxis.set_ticks(np.arange(2.5,4.0,0.5))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [110]:
for vz in [2.5,3.0,3.5]:
    df = policy_avg_df[policy_avg_df.vz_d==vz]

    fig = plt.figure()
    ax = fig.add_subplot(111)
    std = df['G1','std']
    ax.errorbar(df['vx_d'],df['G1','mean'],yerr=std,linestyle='None',marker='o', capsize=5, capthick=1, ecolor='black')


    ax.set_xlabel('Vx_d [m/s]')
    ax.set_ylabel('G1')
    ax.set_title(f'G1 vs Vx_d | Vz_d = {vz} [m/s]')

    ax.set_ylim(0,20)
    ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# G2 Data

In [51]:
## G2 AVG DATA
G2_df = policy_avg_df[['vz_d','vx_d','G2']]
G2_df


fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(G2_df['vx_d'],G2_df['vz_d'],G2_df['G2','mean'],color='red')
ax.set_title('G2 per Trial (Avg)')
ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('G2')

ax.yaxis.set_ticks(np.arange(2.5,4.0,0.5))
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [93]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(policy_df['vx_d'],policy_df['vz_d'],policy_df['G2'])

ax.set_title('G2 per Trial (Raw)')
ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('G2')

ax.set_zlim(0,15)
ax.yaxis.set_ticks(np.arange(2.5,4.0,0.5))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [111]:
for vz in [2.5,3.0,3.5]:
    df = policy_avg_df[policy_avg_df.vz_d==vz]

    fig = plt.figure()
    ax = fig.add_subplot(111)
    std = df['G2','std']
    ax.errorbar(df['vx_d'],df['G2','mean'],yerr=std,linestyle='None',marker='o', capsize=5, capthick=1, ecolor='black')


    ax.set_xlabel('Vx_d [m/s]')
    ax.set_ylabel('G2')
    ax.set_title(f'G2 vs Vx_d | Vz_d = {vz} [m/s]')

    ax.set_ylim(-5,15)
    ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [108]:
policy_avg_df.describe()

vz_d       vx_d landing_rate            RREV_trigger             \
                                   mean        std         mean        std   
count  21.00000  21.000000    21.000000  21.000000    21.000000  21.000000   
mean    3.00000   1.500000     0.584921   0.105416     5.000476   0.152303   
std     0.41833   1.024695     0.218766   0.068909     0.180516   0.057422   
min     2.50000   0.000000     0.183333   0.000000     4.666667   0.043589   
25%     2.50000   0.500000     0.466667   0.050000     4.896667   0.111505   
50%     3.00000   1.500000     0.600000   0.086603     4.970000   0.160416   
75%     3.50000   2.500000     0.783333   0.144338     5.113333   0.180278   
max     3.50000   3.000000     0.900000   0.264575     5.370000   0.277549   

              G1                    G2             ...       wy_f             \
            mean        std       mean        std  ...       mean        std   
count  21.000000  21.000000  21.000000  21.000000  ...  21.000000  21.000000   
mean    7.901270   1.500242   3.423651   1.302173  ... -14.203413   1.775715   
std     1.390025   0.876595   2.293952   1.421778  ...   9.834524   1.111075   
min     4.560000   0.615332   0.363333   0.181475  ... -26.088459   0.451622   
25%     7.116667   0.820183   2.306667   0.499600  ... -21.458107   0.927584   
50%     7.770000   1.198096   3.080000   1.068847  ... -17.646727   1.594788   
75%     8.496667   1.805279   4.183333   1.299654  ... -11.335325   2.097647   
max    11.450000   3.899013  12.040000   6.953359  ...   9.456595   4.332421   

            wz_f                  vx_f                  vy_f             \
            mean        std       mean        std       mean        std   
count  21.000000  21.000000  21.000000  21.000000  21.000000  21.000000   
mean   -1.852194   0.669703   1.225726   0.054424  -0.036873   0.035182   
std     1.141163   0.448965   0.848680   0.053669   0.040988   0.021146   
min    -4.450527   0.124081  -0.036794   0.001708  -0.098321   0.003578   
25%    -2.605606   0.360071   0.484400   0.013499  -0.070039   0.018748   
50%    -1.586450   0.576256   1.006251   0.043859  -0.047947   0.031166   
75%    -1.225739   0.831427   1.872702   0.079481   0.000758   0.049058   
max    -0.085851   1.987885   2.741044   0.221544   0.034083   0.085060   

            vz_f             
            mean        std  
count  21.000000  21.000000  
mean    2.231022   0.088971  
std     0.619529   0.109171  
min     1.017624   0.002862  
25%     2.136994   0.021856  
50%     2.434033   0.061591  
75%     2.624011   0.085922  
max     2.927359   0.397141  

[8 rows x 34 columns]

# Omega_d per Trial

In [112]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(policy_df['vx_d'],policy_df['vz_d'],policy_df['wy_d'])

ax.set_title('Wy_d per Trial (Raw)')
ax.set_xlabel('vx_d [m/s]')
ax.set_ylabel('vz_d [m/s]')
ax.set_zlabel('Wy_d [rad/s]')

ax.yaxis.set_ticks(np.arange(2.5,4.0,0.5))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [122]:
for vz in [2.5,3.0,3.5]:
    df = policy_avg_df[policy_avg_df.vz_d==vz]

    fig = plt.figure()
    ax = fig.add_subplot(111)
    std = df['wy_d','std']
    ax.scatter(df['vx_d'],df['wy_d','mean'],label='wy_d',marker='o',color='black')

    ax.scatter(df['vx_d'],df['wy_f','mean'],label='wy_flip',marker='o',color='red')


    ax.set_xlabel('Vx_d [m/s]')
    ax.set_ylabel('Wy_d [rad/s]')
    ax.set_title(f'Wy_d vs Vx_d | Vz_d = {vz} [m/s]')
    ax.legend()

    
    ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [116]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(df['vx_d'],df['wy_d'])

ax.set_title('wy_d per Trial (Raw)')
ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.grid()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: x and y must be the same size

# Omega_actual per Trial

# Vz_actual per Trial